In [1]:
!pip install python-dotenv langchain-openai langchain-core langchain-community langchain-text-splitters faiss-cpu pymupdf


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import json
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

In [7]:
import os
from dotenv import load_dotenv
from langsmith import Client
from langchain_core.tracers import LangChainTracer

# .env 파일 로드
load_dotenv()

# ✅ 환경 변수 불러오기 상태 확인
print("✅ OpenAI 키 로드됨:", os.getenv("OPENAI_API_KEY") is not None)
print("✅ LangSmith 키 로드됨:", os.getenv("LANGSMITH_API_KEY") is not None)

# LangSmith 환경 설정 (동적 설정)
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")  # .env에서 불러옴
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT") or "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Test"  # 원하는 프로젝트 이름

# LangSmith 클라이언트 직접 사용할 수도 있음
client = Client()
print("현재 LangSmith 프로젝트:", os.environ["LANGCHAIN_PROJECT"])


✅ OpenAI 키 로드됨: True
✅ LangSmith 키 로드됨: True
현재 LangSmith 프로젝트: Test


In [11]:
# 1. 절대경로 지정
absolute_path = r"C:\Users\user\vscode\langchain\data\gov24_serviceList_all.json"

# 2. 파일 존재 여부 확인
if os.path.exists(absolute_path):
    print("✅ 파일 경로 확인 완료:", absolute_path)
else:
    print("❌ 경로에 파일이 존재하지 않습니다.")

# 3. JSON 로드 함수에 직접 경로 넘기기
def load_json_from_absolute_path(file_path: str):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f"📦 JSON 로드 완료: 항목 수 {len(data)}개")
        return data
    except Exception as e:
        print(f"❌ 파일 로드 오류: {e}")
        return []

# 사용 예시
data = load_json_from_absolute_path(absolute_path)

✅ 파일 경로 확인 완료: C:\Users\user\vscode\langchain\data\gov24_serviceList_all.json
📦 JSON 로드 완료: 항목 수 10245개


In [ ]:
from langchain_core.documents import Document

documents = []
for item in data:
    content = f"""
서비스명: {item.get('서비스명')}
서비스목적: {item.get('서비스목적요약')}
지원대상: {item.get('지원대상')}
지원내용: {item.get('지원내용')}
신청방법: {item.get('신청방법')}
신청기한: {item.get('신청기한')}
선정기준: {item.get('선정기준')}
서비스분야: {item.get('서비스분야')}
소관기관: {item.get('소관기관명')}
문의전화: {item.get('전화문의')}
상세조회URL: {item.get('상세조회URL')}
"""
    documents.append(Document(page_content=content.strip(), metadata={"서비스ID": item.get("서비스ID")}))

print(f"LangChain 문서 변환 완료: {len(documents)}개")


📝 LangChain 문서 변환 완료: 10245개


In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [ ]:
split_documents = text_splitter.split_documents(documents)
print(f"분할된 문서 수: {len(split_documents)}")
print("첫 청크 내용:\n", split_documents[0].page_content[:500])

🔹 분할된 문서 수: 11721
🔹 첫 청크 내용:
 서비스명: 유아학비 (누리과정) 지원
서비스목적: 유치원에 다니는 만 3~5세 아동에게 유아학비, 방과후과정비 등 지원
지원대상: ○ 지원대상 : 국공립 및 사립유치원에 다니는 3~5세 유아
  
   - '22년 1~2월생으로 유치원 입학을 희망하여 3세반에 취원한 유아도 지원 대상
   -  취학대상 아동('18.1.1~12.31.출생)이 취학을 유예하는 경우, 유예한 1년에 한하여 5세 유아 무상교육비 지원(취학유예 통지서 제출)
   ※ 단, 지원기간은 3년을 초과할 수 없음.



○ 추가지원 : 저소득층 유아(유아학비 지원 대상 자격이 있고, 사립유치원에 다니는 법정저소득층(기초생활수급자, 차상위계층, 한부모 가정) 유아)

○  아래의 경우 지원대상에서 제외
   -  대한민국 국적을 가지지 않은 유아(난민 및 「재한외국인 처우 기본법」에 따라 법무부장관이 인정한 '특별기여자 등'은 예외적으로 인정)
   - 가정 양육수당 및 어린이집 보육료를 지원


In [15]:
for i, doc in enumerate(split_documents[:3]):
    print(f"\n--- 청크 {i+1} ---")
    print(doc.page_content)


--- 청크 1 ---
서비스명: 유아학비 (누리과정) 지원
서비스목적: 유치원에 다니는 만 3~5세 아동에게 유아학비, 방과후과정비 등 지원
지원대상: ○ 지원대상 : 국공립 및 사립유치원에 다니는 3~5세 유아
  
   - '22년 1~2월생으로 유치원 입학을 희망하여 3세반에 취원한 유아도 지원 대상
   -  취학대상 아동('18.1.1~12.31.출생)이 취학을 유예하는 경우, 유예한 1년에 한하여 5세 유아 무상교육비 지원(취학유예 통지서 제출)
   ※ 단, 지원기간은 3년을 초과할 수 없음.



○ 추가지원 : 저소득층 유아(유아학비 지원 대상 자격이 있고, 사립유치원에 다니는 법정저소득층(기초생활수급자, 차상위계층, 한부모 가정) 유아)

○  아래의 경우 지원대상에서 제외
   -  대한민국 국적을 가지지 않은 유아(난민 및 「재한외국인 처우 기본법」에 따라 법무부장관이 인정한 '특별기여자 등'은 예외적으로 인정)
   - 가정 양육수당 및 어린이집 보육료를 지원 받고 있는 유아
   -  유치원 이용시간에 아이돌봄서비스 등과 중복지원 불가
   - 해외 체류 기간이 31일째 되는 날 유아학비 지원자격 중지

○  자격 중지 후 유아학비를 다시 지원받기 위해서는 재신청 필요, 신청 누락으로 발생되는 지원금은 소급지원 되지 않음.
지원내용: ○ 3~5세에 대해 교육비를 지급합니다.
  - 국공립 100,000원, 사립 280,000원

○ 3~5세에 대해 방과후과정비를 지급합니다.
   - 국공립 50,000원, 사립 70,000원

--- 청크 2 ---
- 국공립 50,000원, 사립 70,000원

○ 사립유치원을 다니는 법정저소득층 유아에게 저소득층 유아학비를 추가 지급합니다.
   - 사립 200,000원
신청방법: 기타 온라인신청||방문신청
신청기한: 상시신청
선정기준: ※ 2025. 3. 1~2026.2.28. 까지 적용

○ 지원대상 : 국공립유치원 및 사립유치원에 다니는  만 3~5세 아동
       5세  '19.1.1

In [16]:
# OpenAIEmbeddings는 OPENAI_API_KEY를 자동으로 .env에서 불러옴
embeddings = OpenAIEmbeddings()

In [17]:
!pip install tiktoken


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import tiktoken

def estimate_embedding_cost(docs, model="text-embedding-3-small", price_per_1k=0.00002):
    """
    문서 리스트에 대한 총 토큰 수 및 예상 비용 계산

    Args:
        docs: LangChain Document 리스트
        model: 사용할 임베딩 모델명 (기본: text-embedding-3-small)
        price_per_1k: 1K 토큰당 비용 (달러)

    Returns:
        total_tokens, estimated_cost
    """
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")  # 대부분 동일 토크나이저 사용
    total_tokens = sum(len(encoding.encode(doc.page_content)) for doc in docs)
    estimated_cost = (total_tokens / 1000) * price_per_1k
    return total_tokens, estimated_cost

# 사용 예시
tokens, cost = estimate_embedding_cost(split_documents)
print(f"🧮 총 토큰 수: {tokens:,}")
print(f"💸 예상 임베딩 비용: ${cost:.6f} USD")

🧮 총 토큰 수: 4,816,550
💸 예상 임베딩 비용: $0.096331 USD


In [19]:
# 문서 임베딩 후 FAISS 저장소에 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

print("FAISS 벡터스토어 생성 완료!")

FAISS 벡터스토어 생성 완료!


In [29]:
retriever_sim = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
retriever_mmr = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.8})

In [30]:
prompt = PromptTemplate.from_template("""
You are an assistant for answering questions about Korean government support policies.
Use the following retrieved context to answer the user's question.
If the answer is not in the context, say "잘 모르겠습니다."
Respond in Korean.

# Context:
{context}

# Question:
{question}

# Answer:
""")

In [31]:
# LangSmith 트레이싱은 .env 설정만으로 자동 활성화됨
# LANGSMITH_TRACING=true 설정 시 실행 로그를 LangSmith에서 확인 가능

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [32]:
chain_sim = (
    {"context": retriever_sim, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain_mmr = (
    {"context": retriever_mmr, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "현재 구직중인 청년이 받을 수 있는 정부지원사업 뭐가있어?"

response_sim = chain_sim.invoke(question)
response_mmr = chain_mmr.invoke(question)

print("🔹 similarity 방식 응답:\n", response_sim)
print("\n🔸 mmr 방식 응답:\n", response_mmr)


🔹 similarity 방식 응답:
 현재 구직 중인 청년이 받을 수 있는 정부지원사업은 다음과 같습니다:

1. **청년 취업활동수당**: 영광군에 거주하는 만 18세~45세 미취업 청년에게 매월 50만원씩 최대 6개월 동안 지원합니다. (문의: 인구교육정책실/061-350-5197)

2. **청년 취업지원 희망프로젝트**: 15세~39세 이하 미취업 청년을 채용한 도내 중소기업에 1인당 월 50~70만원 최대 2년간 지원합니다. (문의: 경제일자리과/064-710-3795)

3. **취업지망생 구직활동지원금 지원**: 양구군 거주 만18세~39세 미취업 청년에게 매월 최대 40만원을 3개월 범위 내에서 지원합니다. (문의: 경제체육과/033-480-7131)

4. **청년 취업활동지원금 지급**: 의왕시에 거주하는 기준중위소득 150% 이하의 미취업 청년에게 취업활동 비용 실비를 지원합니다. (문의: 의왕시청 기업일자리과/031-345-2714)

5. **청년도전 지원사업**: 천안시에 거주하는 만 18세~39세 미취업 청년에게 맞춤형 프로그램을 제공하며, 프로그램 이수 시 참여수당 및 인센티브를 제공합니다. (문의: 천안청년센터 이음/041-900-2031)

각 사업의 상세한 내용과 신청 방법은 해당 기관에 문의하시기 바랍니다.

🔸 mmr 방식 응답:
 잘 모르겠습니다.


In [27]:
question = "3세 이하의 자녀가 있는데 지원가능한 정부지원사업 알려줘?"

response_sim = chain_sim.invoke(question)
response_mmr = chain_mmr.invoke(question)

print("🔹 similarity 방식 응답:\n", response_sim)
print("\n🔸 mmr 방식 응답:\n", response_mmr)


🔹 similarity 방식 응답:
 잘 모르겠습니다.

🔸 mmr 방식 응답:
 3세 이하의 자녀를 위한 지원 가능한 정부지원사업은 다음과 같습니다:

1. **셋째아 이상 아이사랑 건강관리비 지원**
   - 지원대상: 김해시에 거주하는 셋째아 이상 아동 중 만3세 이하의 아동 중 다누리카드 발급자
   - 지원내용: 월 2만원 현금 지원
   - 신청방법: 방문신청
   - 문의전화: 김해시 아동청소년과/055-330-6756

2. **서산시 둘째이후 영유아 양육비 지원**
   - 지원대상: 둘째 이후 자녀로 출생일을 기준으로 만3세 이하인 자녀이며 서산시 관내에 주민등록을 두고 실제로 거주하고 있는 출산 가정
   - 지원내용: 3세(0~35개월)까지 매월 10만원 지원
   - 신청방법: 방문신청
   - 문의전화: 서산시 기획예산담당관/041-660-2149

이 두 가지 지원사업이 3세 이하 자녀를 위한 지원에 해당합니다.


In [28]:
question = "아직 유치원을 가지않은 자녀가 있는데 지원가능한 정부지원사업 알려줘?"

response_sim = chain_sim.invoke(question)
response_mmr = chain_mmr.invoke(question)

print("🔹 similarity 방식 응답:\n", response_sim)
print("\n🔸 mmr 방식 응답:\n", response_mmr)

🔹 similarity 방식 응답:
 잘 모르겠습니다.

🔸 mmr 방식 응답:
 잘 모르겠습니다.
